In [45]:
import torch
import os
import numpy as np
import sys

# TODO: CHANGE AS NEEDED
# tb_path = os.path.realpath("../TreeBERT")
# sys.path.append(tb_path)
# os.chdir(tb_path)

from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AutoTokenizer

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = RobertaModel.from_pretrained("microsoft/codebert-base")
model:RobertaModel = model
model.to(device)
model.add_memory_hooks()

In [49]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [50]:
def get_tensor_size(tensor_):
    return tensor_.nelement() * tensor_.element_size()

def print_tensor_size(tensor_, in_mbs = False):
    print(f'{size_ if (size_:=get_tensor_size(tensor_)) and not in_mbs else size_ / (1024**2):,}')

### Read labels

In [51]:
import pandas as pd
import json

labels = open('pretrain_data_code/java_custom_label/bert_output.json', 'r').readlines()

smells = []

for i in labels:
    test_line = json.loads(i)
    smells.append(test_line["smellKey"])

label_series = pd.Series(smells)

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

ohe = OneHotEncoder(sparse_output=False)
le = LabelEncoder()

label_numpy = ohe.fit_transform(label_series.to_numpy().reshape(-1, 1))
le.fit_transform(label_series.to_numpy())
label_count = len(ohe.categories_[0])

In [64]:
import json
import tqdm
import re
token_list = []

lines = open("../TreeBERT/pretrain_data_code/java_custom_label/bert_output.json").readlines()

lines = tqdm.tqdm(lines, total=len(lines))
i = 0
for line in lines:
    line = re.sub(r" {2,}|\\n|\\r", "", json.loads(line)["function"])
    tokens = [tokenizer.cls_token] + tokenizer.tokenize(line) + [tokenizer.eos_token]
    i += 1
    tokens_ids = np.asarray(tokenizer.convert_tokens_to_ids(tokens))
     # xxxxxxxxxxx
    # https://arxiv.org/abs/1905.05583
    if len(tokens_ids) >= 512:
        # halfway = len(tokens_ids) // 2
        tokens_ids = tokens_ids[:511]
    else:
        tokens_ids = np.pad(tokens_ids, (0, 511 - tokens_ids.shape[0])) 

    tensor = torch.tensor(tokens_ids).to(device)[None, :]
    
    # BaseModelOutputWithPoolingAndCrossAttentions
    token_list.append(tensor)

# token_list = [i["pooler_output"] for i in token_list]
# token_list = [i.reshape(-1, *tuple(i.shape)) for i in token_list]
# [i.shape for i in token_list]

100%|██████████| 7974/7974 [00:07<00:00, 1069.99it/s]


In [65]:
tokens

['<s>',
 'public',
 'Ġlong',
 'Ġget',
 'W',
 'ake',
 'Up',
 '(){',
 'č',
 'Ċ',
 'return',
 'Ġwake',
 'Up',
 ';',
 'č',
 'Ċ',
 '}',
 '</s>']

In [66]:
# [i.shape for i in token_list]
# token_list = [i.reshape(i.shape[0]) for i in token_list]
# [i.shape for i in token_list]
token_tensor = torch.vstack(token_list)
# token_tensor = token_tensor.reshape(-1, 9, 513)
# embeddings = model(token_tensor)

In [67]:
import torch.utils.data as data_utils

batch_size = 32

label_tensor = torch.tensor(label_numpy, dtype=torch.long, device=device)

ds = data_utils.TensorDataset(token_tensor, label_tensor)

loader = data_utils.DataLoader(ds, batch_size=batch_size, shuffle=True)

In [68]:
iterator = tqdm.tqdm(enumerate(loader), total=len(loader))

embeds = []
for i, j in iterator:
    with torch.no_grad():
        embeds.append(model(j[0])["pooler_output"])
    


100%|██████████| 250/250 [01:17<00:00,  3.23it/s]


In [69]:
embeds_tensor = torch.vstack(embeds)

embeds_numpy = embeds_tensor.cpu().detach().numpy()
os.chdir("../general")
embeds_numpy.tofile(open("data/codebert_embeds", "w"))


In [59]:
np.fromfile(open("../general/data/codebert_embeds", "r"), dtype=np.float32).shape


(6124032,)